#Word Embedding - Tokenizer

Embedding(input_dim, output_dim, input_length)

input_dim = Total de palabras en el vocabulario/diccionario. Debe estar indexado, i.e., 1, 2, 3,...

output_dim = Dimensión del espacio vectorial en el cual serán embebidas cada una de las palabras del vocabulario.

input_length = Longitud de la secuencia de palabras de entrada de cada documento.

Recuerda que el proceso de Word-Embedding se puede ver como una proyección: un vector de entrada de un espacio dicretizado de palabras, es PROYECTADO a un espacio continuo de vectores de dimensión fija. Dicha proyección de una palabra dada "w", está basada en todas aquellas palabras que rodean a "w" en todos los documentos en que aparece.  

#Ejemplo:

In [1]:
import numpy as np
from keras.preprocessing.text import Tokenizer   
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding


Using TensorFlow backend.


Definamos 10 documentos, 5 positivos y 5 negativos:

In [0]:
docs = [ 'Muy bien hecho lo hecho',
        'Excelente trabajo muchacho',
        'Sigue como hasta ahora',
        'Lo hiciste muy bien',
        'Que siga el buen ánimo',
        'Pésimo trabajo',
        'Que mal desempeño',
        'Muy mal hecho',
        'Que trabajo tan pobre',
        'No está bien']

In [3]:
for d in docs:
  print(d)

Muy bien hecho lo hecho
Excelente trabajo muchacho
Sigue como hasta ahora
Lo hiciste muy bien
Que siga el buen ánimo
Pésimo trabajo
Que mal desempeño
Muy mal hecho
Que trabajo tan pobre
No está bien


Definamos las etiquetas de ambas clases, positivas y negativas:

In [0]:
labels = np.array([1,1,1,1,1, 0,0,0,0,0])

# Tokenization:

Se requiere que las palabras/tokens de cada documento estén indexados. Esto lo podemos hacer con la clase Tokenizer de Keras. Puedes definir el número de variables, num_words, mayor al total de palabras de vocabulario si no sabes cuántas puedes tener, es mejor tener un valor mayor para evitar colisiones.


In [0]:
n_words = 40

In [0]:
tokens = Tokenizer(num_words=n_words, filters='!"#$%&()*+,-./:;<=>?...', lower=True, split=' ', oov_token='#')

In [0]:
tokens.fit_on_texts(docs)

Veamos el diccionario generado de cada palabra con su índice:

In [8]:
print(tokens.word_index)

{'#': 1, 'muy': 2, 'bien': 3, 'hecho': 4, 'trabajo': 5, 'que': 6, 'lo': 7, 'mal': 8, 'excelente': 9, 'muchacho': 10, 'sigue': 11, 'como': 12, 'hasta': 13, 'ahora': 14, 'hiciste': 15, 'siga': 16, 'el': 17, 'buen': 18, 'ánimo': 19, 'pésimo': 20, 'desempeño': 21, 'tan': 22, 'pobre': 23, 'no': 24, 'está': 25}


Podemos obtener el índice de una palabra del vocabulario:

In [9]:
tokens.word_index['siga']

16

O bien, obtener la palabra de un índice particular:

In [35]:
tokens.index_word[9]

'excelente'

Indexamos cada documento:

In [0]:
encoded_docs= tokens.texts_to_sequences(docs)

In [12]:
print(encoded_docs)

[[2, 3, 4, 7, 4], [9, 5, 10], [11, 12, 13, 14], [7, 15, 2, 3], [6, 16, 17, 18, 19], [20, 5], [6, 8, 21], [2, 8, 4], [6, 5, 22, 23], [24, 25, 3]]


Verificamos el total de palabras en nuestro vocabulario:

In [13]:
vocab_size = len(tokens.word_index) + 1
print(vocab_size)

26


Generamos ahora las secuencias de longitud determinada de cada documento :

Se completa con ceros cuando es menor la longitud del documento, o bien, se trunca cuando es mayor la longitud del documento. En este ejemplo usemos una longitud de 6:

In [0]:
sequences_length = 6

El argumento padding={pre, post}, completa con ceros antes o después de cada secuencia:

In [0]:
padded_docs = pad_sequences(encoded_docs, maxlen=sequences_length, padding='post')

In [16]:
print(padded_docs)

[[ 2  3  4  7  4  0]
 [ 9  5 10  0  0  0]
 [11 12 13 14  0  0]
 [ 7 15  2  3  0  0]
 [ 6 16 17 18 19  0]
 [20  5  0  0  0  0]
 [ 6  8 21  0  0  0]
 [ 2  8  4  0  0  0]
 [ 6  5 22 23  0  0]
 [24 25  3  0  0  0]]


#Modelo:

En este ejemplo generemos vectores de cada palabra continuos de dimensión 4:

In [0]:
embedded_vector_sz=4

In [0]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedded_vector_sz, input_length=sequences_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Observemos en el resumen del modelo que el tamaño de la matriz Embedding W es de 6x4, i.e., entra un vector/sequence "u" de longitud 6 y sale uno de longitud 4:  u^t*W. Sin embargo, contiene a su vez la información de la transformación de los 26 tokens de nuestro vocabulario al vector continuo 4 dimensional.

In [20]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 4)              104       
_________________________________________________________________
flatten_1 (Flatten)          (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.fit(padded_docs, labels, epochs=40, verbose=0)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [22]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)

10/10 [==============================] - 0s 2ms/step


In [23]:
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


#Word Embedding Matrix

Para recuperar la matriz WordEmbeddig debemos utilizar únicamente la primera capa del modelo anterior:

In [0]:
matrixEmbedding = Model(inputs=model.input, outputs=model.get_layer('embedding_1').output)

In [25]:
matrixEmbedding.output_shape

(None, 6, 4)

Y a su vez obtener los vectores continuos 4-dimensionales para cada una de los 26 tokens de nuestro vocabulario:

In [26]:
matrixEmbedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(26, 4) dtype=float32, numpy=
 array([[-0.04464644, -0.05516564,  0.00996508, -0.02029327],
        [-0.00343312,  0.0278131 , -0.00697067, -0.04113562],
        [-0.0008171 ,  0.08894813,  0.07018009, -0.00795872],
        [ 0.080949  ,  0.01199691, -0.03984289, -0.0675302 ],
        [ 0.04170085,  0.06890322,  0.07722773,  0.01145003],
        [-0.00337593,  0.02143945, -0.006995  , -0.07491899],
        [ 0.01655192, -0.07871365,  0.08654396, -0.07193401],
        [ 0.01155774,  0.06323144,  0.00806769,  0.06426584],
        [-0.06300905,  0.02952315,  0.01216486, -0.00665407],
        [-0.00523054,  0.07882133, -0.00823868,  0.05557589],
        [ 0.00184952,  0.08266567,  0.06707029, -0.00067254],
        [-0.08322014,  0.07135184, -0.04240251,  0.0244072 ],
        [ 0.04049359, -0.00655989, -0.03818602,  0.0205704 ],
        [-0.03724208,  0.0182146 , -0.00022752,  0.0137044 ],
        [ 0.0563774 ,  0.05503864, -0.02534409, -0.0157

El oreden está dado en nuestro diccionario tiene 25 tokens, más el primer índice 0 que no está asociado a ningún token (por eso la longitud 26):

In [27]:
print(tokens.word_index)

{'#': 1, 'muy': 2, 'bien': 3, 'hecho': 4, 'trabajo': 5, 'que': 6, 'lo': 7, 'mal': 8, 'excelente': 9, 'muchacho': 10, 'sigue': 11, 'como': 12, 'hasta': 13, 'ahora': 14, 'hiciste': 15, 'siga': 16, 'el': 17, 'buen': 18, 'ánimo': 19, 'pésimo': 20, 'desempeño': 21, 'tan': 22, 'pobre': 23, 'no': 24, 'está': 25}


Recuerda que en realidad en este ejemplo son 24 palabras, más el símbolo "#" que es el índice 1. Y el índice 0 no se usa. Por ello es de longitud 26.

Podemos ahora obtener los vectores densos de una secuencia de palabras de longitud 6 o de alguna en particular:

In [28]:
matrixEmbedding.predict(x=np.array([[ 9,  0,  0,  0,  0,  0 ]]))[0][0]  # excelente(9)

array([-0.00523054,  0.07882133, -0.00823868,  0.05557589], dtype=float32)

In [29]:
matrixEmbedding.predict(x=np.array([[ 18,  5,  0,  0,  0,  0 ]]))[0]  # buen(18) trabajo(5)

array([[ 0.03117838,  0.07922836, -0.00703129, -0.04722486],
       [-0.00337593,  0.02143945, -0.006995  , -0.07491899],
       [-0.04464644, -0.05516564,  0.00996508, -0.02029327],
       [-0.04464644, -0.05516564,  0.00996508, -0.02029327],
       [-0.04464644, -0.05516564,  0.00996508, -0.02029327],
       [-0.04464644, -0.05516564,  0.00996508, -0.02029327]],
      dtype=float32)

Es difícil que con tan pocos documentos y vocabulario se alcance a encontrar alguna similaridad entre palabras, pero veamos las siguientes:

In [47]:
matrixEmbedding.predict(x=np.array([[ 3,  18,  20,  24,  0,  0 ]]))[0][0:4]   # bien(3), buen(18), pésimo(20), no(24)

array([[ 0.080949  ,  0.01199691, -0.03984289, -0.0675302 ],
       [ 0.03117838,  0.07922836, -0.00703129, -0.04722486],
       [ 0.03504785, -0.0685937 ,  0.0485291 , -0.03329598],
       [ 0.06047745, -0.08564305,  0.03386466, -0.07442895]],
      dtype=float32)

#Podemos guardar tanto el modelo como la matriz Word-Embedding:

In [0]:
#model.save('mimodelo.h5')

In [0]:
#matrixEmbedding.save_weights('mispesos.h5')